In [1]:
# example of fitting an auxiliary classifier gan (ac-gan)
import math
import numpy as np
import tensorflow as tf
from PIL import Image
from matplotlib import pyplot

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!cd '/content/drive/MyDrive/Projects/Diabetes-CNN' 

In [4]:
SEED = 42
N_CLASSES = 2
LATENT_DIM = 100
IMAGE_SHAPE = (320, 240, 3)
INPUT_PATH = '/content/drive/MyDrive/Projects/Diabetes-CNN/Data/HC/O/Training/'
OUTPUT_PATH = '/content/drive/MyDrive/Projects/Diabetes-CNN/Data/HC/G/'
MODEL_PATH = '/content/drive/MyDrive/Projects/Diabetes-CNN/Models/HC/'

In [5]:
# define the standalone discriminator model
def define_discriminator():
    init = tf.keras.initializers.RandomNormal(stddev=0.02)
    in_image = tf.keras.layers.Input(shape=IMAGE_SHAPE)
    
    fe = tf.keras.layers.Conv2D(32, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(in_image)
    fe = tf.keras.layers.LeakyReLU(alpha=0.2)(fe)
    fe = tf.keras.layers.Dropout(0.3)(fe)
    
    fe = tf.keras.layers.Conv2D(64, (3,3), padding='same', kernel_initializer=init)(fe)
    fe = tf.keras.layers.BatchNormalization()(fe)
    fe = tf.keras.layers.LeakyReLU(alpha=0.2)(fe)
    fe = tf.keras.layers.Dropout(0.3)(fe)
    
    fe = tf.keras.layers.Conv2D(128, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(fe)
    fe = tf.keras.layers.BatchNormalization()(fe)
    fe = tf.keras.layers.LeakyReLU(alpha=0.2)(fe)
    fe = tf.keras.layers.Dropout(0.3)(fe)
    
    fe = tf.keras.layers.Conv2D(256, (3,3), padding='same', kernel_initializer=init)(fe)
    fe = tf.keras.layers.BatchNormalization()(fe)
    fe = tf.keras.layers.LeakyReLU(alpha=0.2)(fe)
    fe = tf.keras.layers.Dropout(0.3)(fe)
    
    fe = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(fe)
    fe = tf.keras.layers.Flatten()(fe)
    out1 = tf.keras.layers.Dense(1, activation='sigmoid')(fe)
    out2 = tf.keras.layers.Dense(N_CLASSES, activation='softmax')(fe)
    
    model = tf.keras.models.Model(in_image, [out1, out2])
    opt = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)
    model.compile(loss=['binary_crossentropy', 'sparse_categorical_crossentropy'], optimizer=opt)
    return model

In [6]:
# define the standalone generator model
def define_generator():
    init = tf.keras.initializers.RandomNormal(stddev=0.02)
    
    in_label = tf.keras.layers.Input(shape=(1,))
    li = tf.keras.layers.Embedding(N_CLASSES, 50)(in_label)
    
    n_nodes = 80 * 60 * 3
    li = tf.keras.layers.Dense(n_nodes, kernel_initializer=init)(li)
    li = tf.keras.layers.Reshape((80, 60, 3))(li)
    in_lat = tf.keras.layers.Input(shape=(LATENT_DIM,))
    
    n_nodes = 80 * 60 * 3 * 256
    gen = tf.keras.layers.Dense(n_nodes, kernel_initializer=init)(in_lat)
    gen = tf.keras.layers.Activation('relu')(gen)
    gen = tf.keras.layers.Reshape((80, 60, 3 * 256))(gen)
    
    # merge image gen and label input
    merge = tf.keras.layers.Concatenate()([gen, li])
    gen = tf.keras.layers.Conv2DTranspose(128, (5,5), strides=(2,2), padding='same', kernel_initializer=init)(merge)
    gen = tf.keras.layers.BatchNormalization()(gen)
    gen = tf.keras.layers.Activation('relu')(gen)
    gen = tf.keras.layers.Conv2DTranspose(3, (5,5), strides=(2,2), padding='same', kernel_initializer=init)(gen)
    out_layer = tf.keras.layers.Activation('tanh')(gen)
    
    model = tf.keras.models.Model([in_lat, in_label], out_layer)
    return model

In [7]:
# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model):
    # make weights in the discriminator not trainable
    for layer in d_model.layers:
        if not isinstance(layer, tf.keras.layers.BatchNormalization):
            layer.trainable = False
    # connect the outputs of the generator to the inputs of the discriminator
    gan_output = d_model(g_model.output)
    # define gan model as taking noise and label and outputting real/fake and label outputs
    model = tf.keras.models.Model(g_model.input, gan_output)
    # compile model
    opt = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)
    model.compile(loss=['binary_crossentropy', 'sparse_categorical_crossentropy'], optimizer=opt)
    return model

In [8]:
# load images
def load_real_samples():
    train_ds = tf.keras.utils.image_dataset_from_directory(
        INPUT_PATH,
        labels='inferred',
        label_mode='int',
        color_mode='rgb',
        batch_size=32,
        image_size=(320, 240),
        shuffle=True,
        seed=SEED,
        validation_split=None,
        subset=None,
        interpolation='bilinear',
        follow_links=False,
        crop_to_aspect_ratio=False
    )
    
    train_labels = []
    train_images = []
    for images, labels in train_ds:
        for i in range(len(images)):
            train_images.append(images[i])
            train_labels.append(labels[i].numpy())
    
    train_labels = np.array(train_labels)
    train_images = np.array(train_images)
    train_images = train_images.astype('float32')
    train_images = (train_images - 127.5) / 127.5
    return [train_images, train_labels]

In [9]:
# select real samples
def generate_real_samples(dataset, n_samples):
    images, labels = dataset
    ix = np.random.randint(0, images.shape[0], n_samples)
    X, labels = images[ix], labels[ix]
    y = np.ones((n_samples, 1))
    return [X, labels], y

In [10]:
# generate points in latent space as input for the generator
def generate_latent_points(n_samples):
    x_input = np.random.randn(LATENT_DIM * n_samples)
    z_input = x_input.reshape(n_samples, LATENT_DIM)
    labels = np.random.randint(0, N_CLASSES, n_samples)
    return [z_input, labels]

In [11]:
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, n_samples):
    z_input, labels_input = generate_latent_points(n_samples)
    images = generator.predict([z_input, labels_input])
    y = np.zeros((n_samples, 1))
    return [images, labels_input], y

In [12]:
# generate samples and save as a plot and save the model
def summarize_performance(step, g_model, n_samples=100):
    [X, _], _ = generate_fake_samples(g_model, n_samples)
    X = X * 127.5 + 127.5
    for i in range(100):
        pyplot.subplot(10, 10, 1 + i)
        pyplot.axis('off')
        pyplot.imshow(X[i, :, :, 0])
    filename1 = MODEL_PATH + 'generated_plot_%04d.png' % (step+1)
    pyplot.savefig(filename1)
    pyplot.close()
    filename2 = MODEL_PATH + 'model_%04d.h5' % (step+1)
    g_model.save(filename2)

In [13]:
# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, n_epochs=100, n_batch=40):
    # calculate the number of batches per training epoch
    bat_per_epo = int(dataset[0].shape[0] / n_batch)
    # calculate the number of training iterations
    n_steps = bat_per_epo * n_epochs
    # calculate the size of half a batch of samples
    half_batch = int(n_batch / 2)
    # manually enumerate epochs
    for i in range(n_steps):
        # get randomly selected 'real' samples
        [X_real, labels_real], y_real = generate_real_samples(dataset, half_batch)
        # update discriminator model weights
        _,d_r1,d_r2 = d_model.train_on_batch(X_real, [y_real, labels_real])
        # generate 'fake' examples
        [X_fake, labels_fake], y_fake = generate_fake_samples(g_model, half_batch)
        # update discriminator model weights
        _,d_f,d_f2 = d_model.train_on_batch(X_fake, [y_fake, labels_fake])
        # prepare points in latent space as input for the generator
        [z_input, z_labels] = generate_latent_points(n_batch)
        # create inverted labels for the fake samples
        y_gan = np.ones((n_batch, 1))
        # update the generator via the discriminator's error
        _,g_1,g_2 = gan_model.train_on_batch([z_input, z_labels], [y_gan, z_labels])
        # summarize loss on this batch
        print('>%d, dr[%.3f,%.3f], df[%.3f,%.3f], g[%.3f,%.3f]' % (i+1, d_r1,d_r2, d_f,d_f2, g_1,g_2))
        # evaluate the model performance every 'epoch'
        if (i+1) % (bat_per_epo * 250) == 0:
            summarize_performance(i, g_model)

In [14]:
discriminator = define_discriminator()
generator = define_generator()
gan = define_gan(generator, discriminator)
dataset = load_real_samples()
train(generator, discriminator, gan, dataset, n_epochs=1000)

Found 160 files belonging to 2 classes.
1/1 [==============================] - 3s 3s/step
>1, dr[0.262,1.672], df[74.058,77.345], g[0.000,35.619]
1/1 [==============================] - 0s 36ms/step
>2, dr[0.000,36.535], df[0.000,13.999], g[33.917,6.423]
1/1 [==============================] - 0s 35ms/step
>3, dr[30.171,3.341], df[13.590,36.435], g[17.711,26.040]
1/1 [==============================] - 0s 36ms/step
>4, dr[10.336,26.057], df[11.435,25.281], g[20.740,1.276]
1/1 [==============================] - 0s 33ms/step
>5, dr[7.661,1.423], df[1.024,9.828], g[15.686,18.781]
1/1 [==============================] - 0s 31ms/step
>6, dr[3.204,13.007], df[0.000,3.679], g[6.133,35.976]
1/1 [==============================] - 0s 32ms/step
>7, dr[0.018,44.588], df[0.071,22.537], g[5.219,5.693]
1/1 [==============================] - 0s 32ms/step
>8, dr[0.574,2.317], df[0.124,15.734], g[8.276,9.720]
1/1 [==============================] - 0s 37ms/step
>9, dr[0.358,9.591], df[0.043,8.173], g[5.252,1

1/1 [==============================] - 0s 38ms/step
>1001, dr[1.651,0.003], df[1.471,0.000], g[17.340,0.000]
1/1 [==============================] - 0s 32ms/step
>1002, dr[0.001,0.000], df[0.000,0.000], g[22.291,0.000]
1/1 [==============================] - 0s 36ms/step
>1003, dr[2.201,0.014], df[5.685,1.740], g[23.879,0.134]
1/1 [==============================] - 0s 31ms/step
>1004, dr[1.575,0.065], df[0.000,0.000], g[20.269,0.000]
1/1 [==============================] - 0s 34ms/step
>1005, dr[0.564,0.000], df[0.348,0.110], g[12.357,0.005]
1/1 [==============================] - 0s 55ms/step
>1006, dr[0.000,0.000], df[2.256,0.000], g[23.929,0.000]
1/1 [==============================] - 0s 41ms/step
>1007, dr[0.060,0.254], df[0.000,0.000], g[27.984,0.000]
1/1 [==============================] - 0s 46ms/step
>1008, dr[2.039,0.001], df[0.848,0.000], g[10.047,0.000]
1/1 [==============================] - 0s 43ms/step
>1009, dr[1.734,0.000], df[11.518,0.010], g[44.798,0.000]
1/1 [=============

1/1 [==============================] - 0s 32ms/step
>2001, dr[0.002,0.000], df[0.001,0.005], g[9.352,0.584]
1/1 [==============================] - 0s 32ms/step
>2002, dr[0.079,0.014], df[0.143,0.744], g[9.880,0.333]
1/1 [==============================] - 0s 34ms/step
>2003, dr[0.001,0.000], df[0.001,0.126], g[12.215,0.143]
1/1 [==============================] - 0s 39ms/step
>2004, dr[0.134,0.000], df[0.061,0.000], g[8.225,0.000]
1/1 [==============================] - 0s 42ms/step
>2005, dr[0.000,0.000], df[0.204,0.008], g[10.926,0.000]
1/1 [==============================] - 0s 51ms/step
>2006, dr[0.587,0.000], df[0.776,0.000], g[12.369,0.000]
1/1 [==============================] - 0s 53ms/step
>2007, dr[0.001,0.000], df[0.000,0.000], g[17.312,0.000]
1/1 [==============================] - 0s 57ms/step
>2008, dr[3.270,0.000], df[8.833,0.000], g[34.442,0.000]
1/1 [==============================] - 0s 56ms/step
>2009, dr[6.436,0.368], df[0.007,0.000], g[4.653,0.021]
1/1 [==================

1/1 [==============================] - 0s 41ms/step
>3001, dr[1.059,0.000], df[2.626,0.000], g[20.640,0.000]
1/1 [==============================] - 0s 32ms/step
>3002, dr[3.401,0.000], df[0.010,0.000], g[1.362,0.000]
1/1 [==============================] - 0s 17ms/step
>3003, dr[0.000,0.000], df[3.367,0.000], g[23.657,0.003]
1/1 [==============================] - 0s 35ms/step
>3004, dr[3.776,0.000], df[0.371,0.000], g[4.126,0.000]
1/1 [==============================] - 0s 45ms/step
>3005, dr[0.280,0.000], df[1.698,0.000], g[21.442,0.000]
1/1 [==============================] - 0s 47ms/step
>3006, dr[3.664,0.000], df[1.291,0.079], g[11.783,0.000]
1/1 [==============================] - 0s 47ms/step
>3007, dr[0.162,0.000], df[0.003,0.000], g[11.049,0.116]
1/1 [==============================] - 0s 42ms/step
>3008, dr[0.096,0.000], df[0.040,0.552], g[7.580,0.167]
1/1 [==============================] - 0s 56ms/step
>3009, dr[0.021,0.032], df[0.022,0.000], g[6.515,0.000]
1/1 [==================

In [15]:
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples, n_class):
	x_input = np.random.randn(latent_dim * n_samples)
	z_input = x_input.reshape(n_samples, latent_dim)
	labels = np.asarray([n_class for _ in range(n_samples)])
	return [z_input, labels]

In [16]:
# create and save a plot of generated images
def save_plot(examples, n_examples):
	for i in range(n_examples):
		pyplot.subplot(math.sqrt(n_examples), math.sqrt(n_examples), 1 + i)
		pyplot.axis('off')
		pyplot.imshow(examples[i, :, :, 0])
	pyplot.show()

In [17]:
# load model
model = None

In [19]:
model = tf.keras.models.load_model(MODEL_PATH + 'model_4000.h5')
latent_dim = 100
n_examples = 256 # must be a square
classes = [0, 1]

for c in classes:
  latent_points, labels = generate_latent_points(latent_dim, n_examples, c)
  X  = model.predict([latent_points, labels])
  for i in range(len(X)):
    f = f'{OUTPUT_PATH}/{c}/acgan_{i+1:04}.png'
    prd = ((X[i, :, :, :] * 127.5) + 127.5)
    arr = prd.astype(np.uint8)
    img = Image.fromarray(arr.transpose(1, 0, 2))
    #img = Image.fromarray(prd.astype('uint8').reshape(320, 240, 3), 'RGB')
    img.save(f)
    #print(X.shape)

8/8 [==============================] - 2s 224ms/step
